# D* Pathfinding Algorithm for Street Networks
## Google Colab Compatible Version

This notebook implements the D* pathfinding algorithm for finding optimal paths in street networks. D* is particularly useful for dynamic pathfinding where the graph may change during execution.

Features:
- Distance, Duration, or Unlevel percentage optimization
- Dynamic pathfinding capabilities
- Path visualization with maps
- Performance metrics

## 1. Install Required Libraries and Setup

In [ ]:
# Install required packages for Google Colab
!pip install matplotlib networkx numpy pandas

# Import libraries
import csv
import heapq
import math
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime
from google.colab import files
import io

## 2. Upload Data File

In [ ]:
# Upload the CSV file
print("Please upload the 'viana_streets_network_named.csv' file:")
uploaded = files.upload()

# Get the filename
csv_filename = list(uploaded.keys())[0]
print(f"Uploaded file: {csv_filename}")

## 3. D* Algorithm Implementation

In [ ]:
class DStarStreets:
    def __init__(self, graph, coordinates, s_start, s_goal, weight_type='distance'):
        self.graph = graph
        self.coordinates = coordinates
        self.s_start = s_start
        self.s_goal = s_goal
        self.weight_type = weight_type
        self.OPEN = []
        self.t = {}  # State tags: NEW, OPEN, CLOSED
        self.h = {}  # Cost-to-goal estimates
        self.k = {}  # Key values for priority queue
        self.b = {}  # Backpointer (parent)
        self.path = []
        self.visited_nodes = set()

    def init(self):
        """Initialize all nodes"""
        for node in self.graph:
            self.t[node] = 'NEW'
            self.h[node] = float('inf')
            self.k[node] = float('inf')
            self.b[node] = None

    def run(self):
        """Main D* algorithm execution"""
        self.init()
        
        # Set goal with cost 0 and insert into OPEN list
        self.h[self.s_goal] = 0
        self.insert(self.s_goal, 0)
        
        # Process states until start is processed or no path exists
        max_iterations = len(self.graph) * 3
        iteration = 0
        
        print(f"Starting D* search from {self.s_start} to {self.s_goal}")
        
        while self.OPEN and iteration < max_iterations:
            iteration += 1
            current_node = self.process_state()
            
            if current_node is None:
                break
                
            # Check if we've processed the start node
            if current_node == self.s_start:
                print(f"Start node processed after {iteration} iterations")
                break
        
        if self.h[self.s_start] == float('inf'):
            print(f"No path found from {self.s_start} to {self.s_goal} after {iteration} iterations.")
            return []
        
        print(f"Path cost to start: {self.h[self.s_start]}")
        self.extract_path()
        if self.path:
            self.calculate_costs()
        return self.path

    def insert(self, node, h_new):
        """Insert node into OPEN list with new h value"""
        if self.t[node] == 'NEW':
            self.k[node] = h_new
        elif self.t[node] == 'OPEN':
            self.k[node] = min(self.k[node], h_new)
        elif self.t[node] == 'CLOSED':
            self.k[node] = min(self.h[node], h_new)
        
        self.h[node] = h_new
        self.t[node] = 'OPEN'
        heapq.heappush(self.OPEN, (self.k[node], node))

    def process_state(self):
        """Process the node with minimum k value"""
        if not self.OPEN:
            return None
        
        current_k, current_node = heapq.heappop(self.OPEN)
        
        # Skip if this is an outdated entry
        if current_k > self.k[current_node]:
            if self.OPEN:  # Prevent infinite recursion
                return self.process_state()
            else:
                return None
        
        self.k[current_node] = float('inf')
        self.t[current_node] = 'CLOSED'
        self.visited_nodes.add(current_node)
        
        # Process neighbors
        for neighbor_data in self.graph.get(current_node, []):
            neighbor = neighbor_data['destination']
            
            # Calculate cost based on weight type
            if self.weight_type == 'distance':
                cost = neighbor_data['distance_meters']
            elif self.weight_type == 'duration':
                cost = neighbor_data['duration_minutes']
            else:  # unlevel
                cost = neighbor_data['unlevel_percent']
            
            if self.t[neighbor] == 'NEW':
                self.b[neighbor] = current_node
                self.insert(neighbor, self.h[current_node] + cost)
            elif (self.b[neighbor] == current_node and 
                  self.h[neighbor] != self.h[current_node] + cost):
                self.insert(neighbor, self.h[current_node] + cost)
            elif (self.b[neighbor] != current_node and 
                  self.h[neighbor] > self.h[current_node] + cost):
                self.b[neighbor] = current_node
                self.insert(neighbor, self.h[current_node] + cost)
        
        return current_node

    def extract_path(self):
        """Extract the optimal path from start to goal using backpointers"""
        if self.h[self.s_start] == float('inf'):
            print("Cannot extract path - start node not reachable")
            self.path = []
            return
        
        # Follow backpointers from start to goal
        current = self.s_start
        path = [current]
        visited_in_path = {current}
        
        max_path_length = len(self.graph)
        
        while current != self.s_goal and len(path) < max_path_length:
            # Find the best neighbor based on computed h values
            best_neighbor = None
            min_total_cost = float('inf')
            
            for neighbor_data in self.graph.get(current, []):
                neighbor = neighbor_data['destination']
                
                # Skip if already in path to avoid cycles
                if neighbor in visited_in_path:
                    continue
                
                if self.weight_type == 'distance':
                    cost = neighbor_data['distance_meters']
                elif self.weight_type == 'duration':
                    cost = neighbor_data['duration_minutes']
                else:  # unlevel
                    cost = neighbor_data['unlevel_percent']
                
                # Calculate total cost through this neighbor
                total_cost = cost + self.h[neighbor]
                
                if total_cost < min_total_cost:
                    min_total_cost = total_cost
                    best_neighbor = neighbor
            
            if best_neighbor is None:
                print(f"Path extraction failed at node {current}")
                break
            
            path.append(best_neighbor)
            visited_in_path.add(best_neighbor)
            current = best_neighbor
        
        if current == self.s_goal:
            self.path = path
            print(f"Successfully extracted path with {len(path)} nodes")
        else:
            print(f"Path extraction incomplete - reached {current}, goal is {self.s_goal}")
            self.path = path  # Return partial path

    def calculate_costs(self):
        """Calculate and display total costs for the found path"""
        if len(self.path) < 2:
            print("Path too short to calculate costs")
            return
            
        total_distance = 0
        total_duration = 0
        total_unlevel = 0
        
        for i in range(len(self.path) - 1):
            current = self.path[i]
            next_node = self.path[i + 1]
            
            # Find the edge data
            edge_found = False
            for neighbor_data in self.graph.get(current, []):
                if neighbor_data['destination'] == next_node:
                    total_distance += neighbor_data['distance_meters']
                    total_duration += neighbor_data['duration_minutes']
                    total_unlevel += neighbor_data['unlevel_percent']
                    edge_found = True
                    break
            
            if not edge_found:
                print(f"Warning: No edge found between {current} and {next_node}")
        
        print(f"\n========================")
        print(f"D* Path Results:")
        print(f"Path length: {len(self.path)} nodes")
        print(f"Nodes explored: {len(self.visited_nodes)}")
        print(f"Total Distance: {total_distance:.2f} meters")
        print(f"Total Duration: {total_duration:.2f} minutes")
        print(f"Average Unlevel: {total_unlevel/max(1, len(self.path)-1):.2f}%")
        print(f"========================")

## 4. Data Loading Functions

In [ ]:
def read_street_csv(file_path):
    """Read the street network CSV and build graph structure"""
    graph = {}
    coordinates = {}
    seen_connections = set()
    
    with open(file_path, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            origin = row['origin']
            destination = row['destination']
            distance = float(row['distance_meters'])
            duration = float(row['duration_minutes'])
            unlevel = float(row['unlevel_percent'])
            lon = float(row['intersect_lon'])
            lat = float(row['intersect_lat'])
            
            # Avoid duplicate connections
            connection_key = tuple(sorted([origin, destination]))
            if connection_key in seen_connections:
                continue
            seen_connections.add(connection_key)
            
            # Initialize graph nodes
            if origin not in graph:
                graph[origin] = []
            if destination not in graph:
                graph[destination] = []
            
            # Add bidirectional edges
            graph[origin].append({
                'destination': destination,
                'distance_meters': distance,
                'duration_minutes': duration,
                'unlevel_percent': unlevel
            })
            
            graph[destination].append({
                'destination': origin,
                'distance_meters': distance,
                'duration_minutes': duration,
                'unlevel_percent': unlevel
            })
            
            # Store coordinates
            coordinates[origin] = {'lon': lon, 'lat': lat}
            coordinates[destination] = {'lon': lon, 'lat': lat}
    
    return graph, coordinates

## 5. Load and Display Data

In [ ]:
# Load the street network data
print("Loading street network data...")
graph, coordinates = read_street_csv(csv_filename)

print(f"Loaded {len(graph)} streets with {sum(len(edges) for edges in graph.values())//2} connections")

# Display first 20 streets
streets = sorted(list(graph.keys()))
print("\nFirst 20 available streets:")
for i, street in enumerate(streets[:20]):
    print(f"{i}: {street}")

if len(streets) > 20:
    print(f"... and {len(streets) - 20} more streets")

## 6. Execute D* Algorithm

In [ ]:
# Function to get user input for start and end streets
def get_street_input(streets):
    """Get user input for start and end streets with validation"""
    print("Available streets:")
    for i, street in enumerate(streets):
        print(f"{i}: {street}")
    
    print("\n" + "="*50)
    
    # Get start street
    while True:
        try:
            start_input = input("Enter the START street (name or index): ").strip()
            
            # Try to parse as index first
            try:
                start_index = int(start_input)
                if 0 <= start_index < len(streets):
                    start_street = streets[start_index]
                    break
                else:
                    print(f"Index must be between 0 and {len(streets)-1}")
                    continue
            except ValueError:
                # Try to find by name
                matching_streets = [s for s in streets if start_input.lower() in s.lower()]
                if len(matching_streets) == 1:
                    start_street = matching_streets[0]
                    break
                elif len(matching_streets) > 1:
                    print(f"Multiple streets match '{start_input}':")
                    for i, street in enumerate(matching_streets):
                        print(f"  {streets.index(street)}: {street}")
                    print("Please be more specific or use the index.")
                    continue
                else:
                    print(f"No street found matching '{start_input}'. Please try again.")
                    continue
        except KeyboardInterrupt:
            print("\nOperation cancelled.")
            return None, None
        except Exception as e:
            print(f"Error: {e}. Please try again.")
            continue
    
    # Get end street
    while True:
        try:
            end_input = input("Enter the END street (name or index): ").strip()
            
            # Try to parse as index first
            try:
                end_index = int(end_input)
                if 0 <= end_index < len(streets):
                    end_street = streets[end_index]
                    break
                else:
                    print(f"Index must be between 0 and {len(streets)-1}")
                    continue
            except ValueError:
                # Try to find by name
                matching_streets = [s for s in streets if end_input.lower() in s.lower()]
                if len(matching_streets) == 1:
                    end_street = matching_streets[0]
                    break
                elif len(matching_streets) > 1:
                    print(f"Multiple streets match '{end_input}':")
                    for i, street in enumerate(matching_streets):
                        print(f"  {streets.index(street)}: {street}")
                    print("Please be more specific or use the index.")
                    continue
                else:
                    print(f"No street found matching '{end_input}'. Please try again.")
                    continue
        except KeyboardInterrupt:
            print("\nOperation cancelled.")
            return None, None
        except Exception as e:
            print(f"Error: {e}. Please try again.")
            continue
    
    # Get optimization criteria
    while True:
        try:
            print("\nOptimization criteria:")
            print("0: Distance (meters)")
            print("1: Duration (minutes)")
            print("2: Unlevel percentage")
            
            criteria_input = input("Enter optimization criteria (0, 1, or 2): ").strip()
            criteria = int(criteria_input)
            
            if criteria in [0, 1, 2]:
                break
            else:
                print("Please enter 0, 1, or 2.")
                continue
        except ValueError:
            print("Please enter a valid number (0, 1, or 2).")
            continue
        except KeyboardInterrupt:
            print("\nOperation cancelled.")
            return None, None
    
    return start_street, end_street, criteria

print("Ready to get user input for streets...")

In [ ]:
# Get user input for start and end streets
print("Please select the start and end streets for pathfinding:")
result = get_street_input(streets)

if result[0] is None or result[1] is None:
    print("Operation cancelled. Please run this cell again to select streets.")
else:
    start_street, end_street, criteria = result
    
    weight_types = ['distance', 'duration', 'unlevel']
    weight_type = weight_types[criteria]
    
    print(f"\nSelected route:")
    print(f"Start: {start_street}")
    print(f"End: {end_street}")
    print(f"Optimization: {weight_type}")
    
    print(f"\nFinding optimal path using D* algorithm...")
    
    # Run D* algorithm
    start_time = datetime.now()
    dstar = DStarStreets(graph, coordinates, start_street, end_street, weight_type)
    path = dstar.run()
    end_time = datetime.now()
    
    if path:
        print(f"\nOptimal path found in {end_time - start_time}:")
        for i, street in enumerate(path):
            print(f"{i+1}: {street}")
    else:
        print("No path found.")

## 7. Path Visualization

In [ ]:
def visualize_path_with_exploration(graph, coordinates, path=None, visited_nodes=None, algorithm_name="D*"):
    """Visualize the street network with path and exploration highlighting"""
    plt.figure(figsize=(14, 10))
    
    # Extract coordinates for plotting
    lons = [coord['lon'] for coord in coordinates.values()]
    lats = [coord['lat'] for coord in coordinates.values()]
    
    # Plot all intersections
    plt.scatter(lons, lats, c='lightblue', s=20, alpha=0.6, label='Intersections')
    
    # Draw edges
    for street, neighbors in graph.items():
        if street in coordinates:
            start_coord = coordinates[street]
            for neighbor_data in neighbors:
                neighbor = neighbor_data['destination']
                if neighbor in coordinates:
                    end_coord = coordinates[neighbor]
                    plt.plot([start_coord['lon'], end_coord['lon']], 
                            [start_coord['lat'], end_coord['lat']], 
                            'lightgray', linewidth=0.5, alpha=0.6)
    
    # Highlight visited nodes (exploration)
    if visited_nodes:
        visited_lons = [coordinates[street]['lon'] for street in visited_nodes if street in coordinates]
        visited_lats = [coordinates[street]['lat'] for street in visited_nodes if street in coordinates]
        plt.scatter(visited_lons, visited_lats, c='orange', s=40, alpha=0.7, label='Explored Nodes', zorder=4)
    
    # Highlight path if provided
    if path and len(path) > 1:
        path_lons = [coordinates[street]['lon'] for street in path if street in coordinates]
        path_lats = [coordinates[street]['lat'] for street in path if street in coordinates]
        
        # Draw path
        plt.plot(path_lons, path_lats, 'red', linewidth=3, alpha=0.8, label=f'{algorithm_name} Path')
        plt.scatter(path_lons, path_lats, c='red', s=50, alpha=0.9, zorder=5)
        
        # Mark start and end
        if path:
            start_coord = coordinates[path[0]]
            end_coord = coordinates[path[-1]]
            plt.scatter(start_coord['lon'], start_coord['lat'], c='green', s=100, 
                       marker='o', label='Start', zorder=6)
            plt.scatter(end_coord['lon'], end_coord['lat'], c='purple', s=100, 
                       marker='s', label='End', zorder=6)
    
    plt.title(f'Viana do Castelo Street Network - {algorithm_name} Algorithm', fontsize=14, weight='bold')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# Visualize the result if path was found
if 'path' in locals() and path:
    print(f"\nVisualizing the D* optimal path from {start_street} to {end_street}...")
    print(f"Path includes {len(path)} streets with {len(dstar.visited_nodes)} nodes explored")
    visualize_path_with_exploration(graph, coordinates, path, dstar.visited_nodes, "D*")
elif 'graph' in locals():
    print("\nNo path found to visualize. Showing the street network...")
    visualize_path_with_exploration(graph, coordinates)
else:
    print("Please run the previous cells to load data and find a path first.")

## 8. Algorithm Comparison

In [ ]:
# Compare D* with different optimization criteria
def compare_optimization_criteria(start_street, end_street):
    """Compare D* performance with different optimization criteria"""
    results = []
    
    for i, weight_type in enumerate(['distance', 'duration', 'unlevel']):
        print(f"\nTesting D* with {weight_type} optimization...")
        
        start_time = datetime.now()
        dstar_comp = DStarStreets(graph, coordinates, start_street, end_street, weight_type)
        path_comp = dstar_comp.run()
        end_time = datetime.now()
        
        if path_comp:
            results.append({
                'optimization': weight_type,
                'path_length': len(path_comp),
                'nodes_explored': len(dstar_comp.visited_nodes),
                'execution_time': (end_time - start_time).total_seconds()
            })
    
    if results:
        df = pd.DataFrame(results)
        print("\n" + "="*60)
        print("D* COMPARISON RESULTS")
        print("="*60)
        print(df.to_string(index=False))
    
    return results

# Run comparison using the same streets selected by the user
if 'start_street' in locals() and 'end_street' in locals():
    print(f"\nRunning comparison for route: {start_street} → {end_street}")
    comparison_results = compare_optimization_criteria(start_street, end_street)
else:
    print("Please run the previous cell to select streets first, then run this cell for comparison.")